In [1]:
import argparse
from enum import Enum
import io
import os
from google.cloud import vision
from google.cloud.vision import types
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import math

In [2]:
class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5

In [3]:
def draw_boxes(image, bounds, color):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'ServiceAccountToken.json'
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)

    for bound in bounds:
        draw.polygon([
            bound.vertices[0].x, bound.vertices[0].y,
            bound.vertices[1].x, bound.vertices[1].y,
            bound.vertices[2].x, bound.vertices[2].y,
            bound.vertices[3].x, bound.vertices[3].y], None, color)
    return image

In [4]:
global k
def get_document_bounds(image_file, feature):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'ServiceAccountToken.json'
    client = vision.ImageAnnotatorClient()

    bounds = []

    with io.open(image_file, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    response = client.document_text_detection(image=image)
    document = response.full_text_annotation
    k=document
    # Collect specified feature bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                            bounds.append(symbol.bounding_box)

                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)

                if (feature == FeatureType.PARA):
                    bounds.append(paragraph.bounding_box)

            if (feature == FeatureType.BLOCK):
                bounds.append(block.bounding_box)

    # The list `bounds` contains the coordinates of the bounding boxes.
    return bounds

In [5]:
def render_doc_text(filein, fileout):
    image = Image.open(filein)
    bounds = get_document_bounds(filein, FeatureType.BLOCK)
    draw_boxes(image, bounds, 'blue')
    bounds = get_document_bounds(filein, FeatureType.PARA)
    draw_boxes(image, bounds, 'red')
    bounds = get_document_bounds(filein, FeatureType.WORD)
    draw_boxes(image, bounds, 'yellow')

    if fileout != 0:
        image.save(fileout)
    else:
        image.show()

In [6]:
if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('detect_file', help='The image for text detection.')
#     parser.add_argument('-out_file', help='Optional output file', default=0)
#     args = parser.parse_args()
    path = os.path.join(r'D:\SiH-Frame-Project\MyVisionDemo','abc4.jpeg')
    render_doc_text(path, 'output-note-med-abc4.jpeg')

In [8]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'ServiceAccountToken.json'
client = vision.ImageAnnotatorClient()

bounds = []

with io.open(os.path.join(r'D:\SiH-Frame-Project\MyVisionDemo','abc6.jpeg'), 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

response = client.document_text_detection(image=image)
document = response.full_text_annotation

In [9]:
sample=document.text.split('\n')

In [10]:
list(filter(lambda k :'Tablets' in k,sample))

[]

In [11]:
 bounds = get_document_bounds(path, FeatureType.PARA)

In [12]:
k=''
for page in document.pages:
    for block in page.blocks:
        for paragraph in block.paragraphs:
            for word in paragraph.words:
                for symbol in word.symbols:
                    k=k+k.join(str(symbol.text))
                k=k+k.join(' ')
            k=k+k.join('\n')

In [13]:
k = k.split('\n')

In [14]:
k

['Next Wave ( in ) RSG Pontes ',
 'Paonta Sahib - 1 ',
 '8025 ',
 'Dis ',
 'NB / 06 / 335 ',
 '8904103308310 ',
 'Mankind Market by : DISCOVERY MANKIND Division of kind Phama ',
 'n Estate 3 New ',
 'ver Trade M S I owners Mankind Prime ',
 'DSP ZOAGER ',
 'o ',
 'Domperidone Prolonged release Capsules IP PANTAKINDDSRT ',
 '1 पेन्टाकडीए ',
 'ORTANİNDD ',
 'SESTA ',
 'RSS ',
 '11 ',
 'Each nard gelin capsule core bentoprazole dium P ',
 'lifeq . to Pantoprole Tras gastoressant pellets Dompendone ',
 'as prolonged - lease pellets Colours For Oude Red US ',
 'proved colours used in ',
 'one ',
 'GEBODERNARD GRANADAN ',
 'um Dioxide IP ',
 'Shell ',
 '. . . : : : 1 ',
 'B ',
 'EL ',
 'not exceeding keep out of reach of children . ',
 '']

In [15]:
list(filter(lambda value:'Tablet' in value or 'capsule' in value,k))

['Each nard gelin capsule core bentoprazole dium P ']

In [16]:
k=''
for page in document.pages:
    for block in page.blocks:
        for paragraph in block.paragraphs:
            for word in paragraph.words:
                for symbol in word.symbols:
                    k=k+k.join(str(symbol.text))
                k=k+k.join(' ')
            k=k+k.join('\t')
        k=k+k.join('\n')
        k=k+k.join('\n')
        k=k+k.join('\n')

In [17]:
k = k.split('\n')

In [117]:
c=open("json-note3.json","w+")

In [118]:
c.writelines(str(document))

In [119]:
k=''
for page in document.pages:
    for block in page.blocks:
        for paragraph in block.paragraphs:
            for word in paragraph.words:
                for symbol in word.symbols:
                    k=k+k.join(str(symbol.text))
                k=k+k.join(' ')
            k=k+k.join('\t')
        k=k+k.join('\n')
        k=k+k.join('\n')
        k=k+k.join('\n')

In [120]:
vertex={}
i=1
for page in document.pages:
    for block in page.blocks:
        list_x_y=[]
        for vertices in block.bounding_box.vertices:
            list_x_y.append(vertices)
        vertex[i]=list_x_y
        i=i+1

In [121]:
list(vertex.keys())

[1, 2, 3, 4, 5, 6]

In [123]:
df=pd.DataFrame(columns=["block","x","y"])
for ver in vertex:
    for i in range(0,4):
        df=df.append(dict(
        block=ver,
        x=vertex[ver][i].x,
        y=vertex[ver][i].y
        ),ignore_index=True)

In [124]:
df

,block,x,y
0,1,313,227
1,1,378,222
2,1,378,231
3,1,314,236
4,2,429,222
5,2,767,209
6,2,770,302
7,2,433,315
8,3,428,322
9,3,772,305


In [129]:
k=''
for paragraph in document.pages[0].blocks[1].paragraphs:
    for word in paragraph.words:
        for symbol in word.symbols:
            k=k+k.join(str(symbol.text))
        k=k+k.join(' ')
    k=k+k.join('\n')

In [130]:
k

'r in Sore Engineer \ne rity holes ( One of them did the ather patch did not work ) . Microsoft d e s back up critical files before installing patches " \nThis situation is quite embarraning and clearly explains the state of Cairs of remanet . The developers were either to rusheder to careless to fix obvious de \nWe may keep on disain the history of software failure which have play human safety and the projects failures in the past \n'

In [131]:
path=os.path.join(r'D:\SiH-Frame-Project\MyVisionDemo','letter-note3.jpeg')
image=Image.open(path)
draw = ImageDraw.Draw(image)
draw.polygon([
    vertex[3][0].x,vertex[3][0].y,
    vertex[3][1].x,vertex[3][1].y,
    vertex[3][2].x,vertex[3][2].y,
    vertex[3][3].x,vertex[3][3].y,], None,'blue')
image.save('custom-output-note3-block4.jpeg')

In [132]:
draw.polygon([
    vertex[4][0].x,vertex[4][0].y,
    vertex[4][1].x,vertex[4][1].y,
    vertex[4][2].x,vertex[4][2].y,
    vertex[4][3].x,vertex[4][3].y,], None,'green')
image.save('custom-output-note3-block4.jpeg')

In [133]:
draw.polygon([
    vertex[5][0].x,vertex[5][0].y,
    vertex[5][1].x,vertex[5][1].y,
    vertex[5][2].x,vertex[5][2].y,
    vertex[5][3].x,vertex[5][3].y,], None,'red')
image.save('custom-output-note3-block4.jpeg')

In [134]:
draw.polygon([
    vertex[1][0].x,vertex[1][0].y,
    vertex[1][1].x,vertex[1][1].y,
    vertex[1][2].x,vertex[1][2].y,
    vertex[1][3].x,vertex[1][3].y,], None,'yellow')
image.save('custom-output-note3-block4.jpeg')

In [135]:
draw.polygon([
    vertex[2][0].x,vertex[2][0].y,
    vertex[2][1].x,vertex[2][1].y,
    vertex[2][2].x,vertex[2][2].y,
    vertex[2][3].x,vertex[2][3].y,], None,'black')
image.save('custom-output-note3-block4.jpeg')

In [136]:
draw.polygon([
    vertex[6][0].x,vertex[6][0].y,
    vertex[6][1].x,vertex[6][1].y,
    vertex[6][2].x,vertex[6][2].y,
    vertex[6][3].x,vertex[6][3].y,], None,'white')
image.save('custom-output-note3-block4.jpeg')

In [137]:
draw.polygon([
    vertex[6][0].x,vertex[6][0].y,0,0], None,'pink')
image.save('custom-output-note3-block4.jpeg')

In [140]:
print(vertex[1][0].x,vertex[1][0].y,)
print(vertex[2][0].x,vertex[2][0].y,)
print(vertex[3][0].x,vertex[3][0].y,)
print(vertex[4][0].x,vertex[4][0].y,)
print(vertex[5][0].x,vertex[5][0].y,)
print(vertex[6][0].x,vertex[6][0].y,)

313 227
429 222
428 322
439 408
35 260
428 423


In [151]:
x_cor=list([vertex[1][0].x,vertex[2][0].x,vertex[3][0].x,vertex[4][0].x,vertex[5][0].x,vertex[6][0].x])

In [152]:
x_cor

[313, 429, 428, 439, 35, 428]

In [153]:
x_cor.sort()

In [154]:
x_cor

[35, 313, 428, 428, 429, 439]

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'ServiceAccountToken.json'
client = vision.ImageAnnotatorClient()

bounds = []



In [ ]:
with io.open(os.path.join(r'D:\SiH-Frame-Project\MyVisionDemo','letter-note3.jpeg'), 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

response = client.document_text_detection(image=image)

In [7]:
features=client._get_all_features()

In [8]:
features

[{'type': <Type.FACE_DETECTION: 1>},
 {'type': <Type.LANDMARK_DETECTION: 2>},
 {'type': <Type.LOGO_DETECTION: 3>},
 {'type': <Type.LABEL_DETECTION: 4>},
 {'type': <Type.TEXT_DETECTION: 5>},
 {'type': <Type.DOCUMENT_TEXT_DETECTION: 11>},
 {'type': <Type.SAFE_SEARCH_DETECTION: 6>},
 {'type': <Type.IMAGE_PROPERTIES: 7>},
 {'type': <Type.CROP_HINTS: 9>},
 {'type': <Type.WEB_DETECTION: 10>},
 {'type': <Type.PRODUCT_SEARCH: 12>},
 {'type': <Type.OBJECT_LOCALIZATION: 19>}]

In [97]:
request = {
             'image': {
                 'source': {'image_uri': 'https://www.crescentpharma.com/wp-content/uploads/2016/01/Paracetamol-500mg-100-CapletsPOM.png'},
            },
         }

In [98]:
response = client.annotate_image(request)

In [25]:
file = open("response.json","w+")

In [28]:
file.writelines(str(response))

In [29]:
type(response)

google.cloud.vision_v1.types.AnnotateImageResponse

In [100]:
document=response.full_text_annotation

In [59]:
frame = pd.DataFrame(columns=['x1','y1','x2','y2','x3','y3','x4','y4','area','text']) 

In [60]:
frame

,x1,y1,x2,y2,x3,y3,x4,y4,area,text


In [ ]:
for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                            bounds.append(symbol.bounding_box)

                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)

                if (feature == FeatureType.PARA):
                    bounds.append(paragraph.bounding_box)

            if (feature == FeatureType.BLOCK):
                bounds.append(block.bounding_box)

In [61]:
x1=0
x2=0
x3=0
x4=0
y1=0
y2=0
y3=0
y4=0
text=''
for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                x1=paragraph.words[0].symbols[0].bounding_box.vertices[0].x
                x2=paragraph.words[0].symbols[0].bounding_box.vertices[1].x
                x3=paragraph.words[0].symbols[0].bounding_box.vertices[2].x
                x4=paragraph.words[0].symbols[0].bounding_box.vertices[3].x
                y1=paragraph.words[0].symbols[0].bounding_box.vertices[0].y
                y2=paragraph.words[0].symbols[0].bounding_box.vertices[1].y
                y3=paragraph.words[0].symbols[0].bounding_box.vertices[2].y
                y4=paragraph.words[0].symbols[0].bounding_box.vertices[3].y
                area=math.sqrt((x2-x1)**2+(y2-y1)**2)*math.sqrt((x3-x2)**2+(y3-y2)**2)
                for word in paragraph.words:
                    print(word.symbols[0].bounding_box)
                    for symbol in word.symbols:
                        text+=text.join(symbol.text)
                    text+=text.join(' ')
                frame=frame.append(dict(x1=x1,
                                  x2=x2,
                                  x3=x3,
                                  x4=x4,
                                  y1=y1,
                                  y2=y2,
                                  y3=y3,
                                  y4=y4,
                                  area=area,
                                  text = text),ignore_index=True)
                text=''
               

vertices {
  x: 65
  y: 77
}
vertices {
  x: 75
  y: 76
}
vertices {
  x: 77
  y: 93
}
vertices {
  x: 67
  y: 94
}

vertices {
  x: 104
  y: 73
}
vertices {
  x: 114
  y: 72
}
vertices {
  x: 116
  y: 89
}
vertices {
  x: 106
  y: 90
}

vertices {
  x: 154
  y: 69
}
vertices {
  x: 158
  y: 69
}
vertices {
  x: 160
  y: 86
}
vertices {
  x: 156
  y: 86
}

vertices {
  x: 158
  y: 69
}
vertices {
  x: 161
  y: 69
}
vertices {
  x: 163
  y: 86
}
vertices {
  x: 160
  y: 86
}

vertices {
  x: 193
  y: 65
}
vertices {
  x: 197
  y: 65
}
vertices {
  x: 199
  y: 82
}
vertices {
  x: 195
  y: 82
}

vertices {
  x: 56
  y: 101
}
vertices {
  x: 86
  y: 98
}
vertices {
  x: 88
  y: 117
}
vertices {
  x: 58
  y: 120
}

vertices {
  x: 170
  y: 92
}
vertices {
  x: 185
  y: 91
}
vertices {
  x: 187
  y: 110
}
vertices {
  x: 172
  y: 111
}

vertices {
  x: 174
  y: 86
}
vertices {
  x: 184
  y: 87
}
vertices {
  x: 182
  y: 107
}
vertices {
  x: 172
  y: 106
}

vertices {
  x: 240
  y: 91
}
ver

In [105]:
document

pages {
  property {
    detected_languages {
      language_code: "en"
      confidence: 0.7799999713897705
    }
  }
  width: 463
  height: 463
  blocks {
    property {
      detected_languages {
        language_code: "en"
        confidence: 1.0
      }
    }
    bounding_box {
      vertices {
        x: 45
        y: 97
      }
      vertices {
        x: 318
        y: 106
      }
      vertices {
        x: 315
        y: 192
      }
      vertices {
        x: 42
        y: 183
      }
    }
    paragraphs {
      property {
        detected_languages {
          language_code: "en"
          confidence: 1.0
        }
      }
      bounding_box {
        vertices {
          x: 45
          y: 97
        }
        vertices {
          x: 318
          y: 106
        }
        vertices {
          x: 315
          y: 192
        }
        vertices {
          x: 42
          y: 183
        }
      }
      words {
        property {
          detected_languages {
            la

In [62]:
frame

,x1,y1,x2,y2,x3,y3,x4,y4,area,text
0,65,77,75,76,77,93,67,94,172.026161,Next Wave ( in ) RSG Pontes
1,174,86,184,87,182,107,172,106,202.000000,Paonta Sahib - 1
2,309,90,320,90,320,111,309,111,231.000000,8025
3,69,125,78,125,77,154,68,154,261.155126,Dis
4,178,129,190,129,189,158,177,158,348.206835,NB / 06 / 335
5,378,186,386,185,390,212,382,213,220.056811,8904103308310
6,133,203,142,203,142,232,133,232,261.000000,Mankind Market by : DISCOVERY MANKIND Division...
7,138,321,143,321,143,343,138,343,110.000000,n Estate 3 New
8,131,347,142,347,142,369,131,369,242.000000,ver Trade M S I owners Mankind Prime
9,519,235,519,244,503,243,503,234,144.280976,DSP ZOAGER


In [81]:
list(frame[frame['area']==max(frame['area'])]['text'])

['Domperidone Prolonged release Capsules IP PANTAKINDDSRT ']

In [69]:
li[0]

11    96
Name: x1, dtype: object